In [2]:
# Imports
import torch
from torch.quantization import get_default_qat_qconfig, prepare_qat, convert
from datasets import load_dataset
import transformers
from transformers import (
    DataCollatorForLanguageModeling,
    AutoModelForCausalLM,
    #MistralForCausalLM,
    AutoTokenizer,
    HfArgumentParser,
    TrainingArguments,
    pipeline,
    logging,
    BitsAndBytesConfig,
)
import bitsandbytes
from peft import LoraConfig, PeftModel
from trl import SFTTrainer
from datasets import concatenate_datasets, load_dataset
import os

In [3]:
# Parameters
# Define the base path in Google Drive to store the model
#base_path = '/content/drive/MyDrive/huggingface_models'

# The model that you want to train from the Hugging Face hub
model_id = "cognitivecomputations/dolphin-2.2.1-mistral-7b"

# The instruction dataset to use
dataset_name = "norygano/TRACHI"

# Fine-tuned model name
new_model = "dolphin-mistral-TRACHI-7b"

# Constants
model_name = model_id.split('/')[-1]

################################################################################
# QLoRA parameters
################################################################################

# LoRA attention dimension
lora_r = 6

# Alpha parameter for LoRA scaling
lora_alpha = 8

# Dropout probability for LoRA layers
lora_dropout = 0.05

################################################################################
# bitsandbytes parameters
################################################################################

# Activate 4-bit precision base model loading
use_4bit = True

# Compute dtype for 4-bit base models
bnb_4bit_compute_dtype = "float16"

# Quantization type (fp4 or nf4)
bnb_4bit_quant_type = "nf4"

# Activate nested quantization for 4-bit base models (double quantization)
use_nested_quant = True

################################################################################
# TrainingArguments parameters
################################################################################

# Output directory where the model predictions and checkpoints will be stored
output_dir = "./results"

# Number of training epochs
num_train_epochs = 3

# Enable fp16/bf16 training (set bf16 to True with an A100)
fp16 = False
bf16 = True

# Batch size per GPU for training
per_device_train_batch_size = 4

# Batch size per GPU for evaluation
per_device_eval_batch_size = 4

# Number of update steps to accumulate the gradients for
gradient_accumulation_steps = 2

# Enable gradient checkpointing
gradient_checkpointing = True

# Maximum gradient normal (gradient clipping)
max_grad_norm = 0.3

# Initial learning rate (AdamW optimizer)
learning_rate = 2e-4

# Weight decay to apply to all layers except bias/LayerNorm weights
weight_decay = 0.001

# Optimizer to use
optim = "paged_adamw_32bit"

# Learning rate schedule
lr_scheduler_type = "cosine"

# Number of training steps (overrides num_train_epochs)
max_steps = -1

# Ratio of steps for a linear warmup (from 0 to learning rate)
warmup_ratio = 0.03

# Group sequences into batches with same length
# Saves memory and speeds up training considerably
group_by_length = True

# Save checkpoint every X updates steps
save_steps = 0

# Log every X updates steps
logging_steps = 5

################################################################################
# SFT parameters
################################################################################

# Maximum sequence length to use
max_seq_length = None

# Pack multiple short examples in the same input sequence to increase efficiency
packing = False

# Load the entire model on the GPU 0
device_map = {"": 0}

In [ ]:
# Fine-Tune
import gc

torch.backends.cuda.matmul.allow_tf32 = True
torch.backends.cudnn.allow_tf32 = True

# Assuming `model` is your model variable
if 'model' in locals():
    del model
    torch.cuda.empty_cache()
    gc.collect()

# Load your dataset
torch.autograd.set_detect_anomaly(True)

# Load tokenizer
tokenizer = AutoTokenizer.from_pretrained(model_id)
#tokenizer.add_special_tokens({'pad_token': '[PAD]'})
#model.resize_token_embeddings(len(tokenizer))
tokenizer.padding_side = "right" # Fix weird overflow issue with fp16 training

dataset = load_dataset(dataset_name, split="train")

# Function to duplicate entries in the dataset
def duplicate_entries(dataset, duplication_factor):
    duplicated_datasets = [dataset for _ in range(duplication_factor)]
    concatenated_dataset = concatenate_datasets(duplicated_datasets)
    return concatenated_dataset.shuffle(seed=42)  # Shuffle to mix the entries

# Increase the weight of the dataset by duplicating its entries
dataset = duplicate_entries(dataset, duplication_factor=3)

#model = AutoModelForCausalLM.from_pretrained(model_name)

# Function to apply chat template to each entry in the dataset
def apply_chat_template(batch):
    # Apply the chat template with `add_generation_prompt=False`
    # Adjust the following line if your data structure is different
    formatted_chats = [tokenizer.apply_chat_template(chat, tokenize=False, add_generation_prompt=False) for chat in batch['chat']]
    return {'formatted_chat': formatted_chats}

# Applying chat template to the dataset
dataset = dataset.map(apply_chat_template, batched=True)

# Tokenize the formatted chats
def tokenize_function(batch):
    # Ensure this line correctly handles your data's structure
    return tokenizer(batch['formatted_chat'], padding=True, truncation=True, max_length=512)

# Applying tokenization
dataset = dataset.map(tokenize_function, batched=True)

# Load tokenizer and model with QLoRA configuration
compute_dtype = getattr(torch, bnb_4bit_compute_dtype)


bnb_config = BitsAndBytesConfig(
    load_in_4bit=use_4bit,
    bnb_4bit_quant_type=bnb_4bit_quant_type,
    bnb_4bit_compute_dtype=compute_dtype,
    bnb_4bit_use_double_quant=use_nested_quant,
)

# Load base model
model = AutoModelForCausalLM.from_pretrained(
    model_id,
    attn_implementation="flash_attention_2",
    quantization_config=bnb_config,
    device_map=device_map
)
model.config.use_cache = False
model.config.pretraining_tp = 1

# Check GPU compatibility with bfloat16
if compute_dtype == torch.float16 and use_4bit:
    major, _ = torch.cuda.get_device_capability()
    if major >= 8:
        print("=" * 80)
        print("Your GPU supports bfloat16: accelerate training with bf16=True")
        print("=" * 80)

# Initialize data collator
data_collator = DataCollatorForLanguageModeling(tokenizer=tokenizer, return_tensors="pt", mlm=False)

# Load LoRA configuration
peft_config = LoraConfig(
    lora_alpha=lora_alpha,
    lora_dropout=lora_dropout,
    r=lora_r,
    bias="none",
    task_type="CAUSAL_LM",
    target_modules=["q_proj", "k_proj", "v_proj", "o_proj","gate_proj"]
)

# Set training parameters
training_arguments = TrainingArguments(
    output_dir=output_dir,
    num_train_epochs=num_train_epochs,
    per_device_train_batch_size=per_device_train_batch_size,
    gradient_accumulation_steps=gradient_accumulation_steps,
    optim=optim,
    save_steps=save_steps,
    logging_steps=logging_steps,
    learning_rate=learning_rate,
    weight_decay=weight_decay,
    fp16=fp16,
    bf16=bf16,
    max_grad_norm=max_grad_norm,
    max_steps=max_steps,
    warmup_ratio=warmup_ratio,
    group_by_length=group_by_length,
    lr_scheduler_type=lr_scheduler_type,
    tf32=True,
    report_to="tensorboard"
)

# Set supervised fine-tuning parameters
trainer = SFTTrainer(
    model=model,
    train_dataset=dataset,
    data_collator=data_collator,
    peft_config=peft_config,
    dataset_text_field="formatted_chat",
    max_seq_length=max_seq_length,
    tokenizer=tokenizer,
    args=training_arguments,
    packing=packing,
)

# Train model
trainer.train()

# Save trained model
trainer.model.save_pretrained(new_model)

# Ignore warnings
#prompt = "Who are you?"
#pipe = pipeline(task="text-generation", model=model, tokenizer=tokenizer, max_length=200)

# Adjust the prompt format
#formatted_prompt = f"### HUMAN:\n{prompt}\n\n### RESPONSE: \n"

# Use the formatted prompt
#result = pipe(formatted_prompt)

# Print the generated text, assuming the model appends its response after "### RESPONSE: \n"
#print(result[0]['generated_text'])

In [ ]:
# Reload (FP16) -> merge w/ LoRA weights

# Cleanup #CUDA-OOM
if 'model' in locals():
  del model
if 'pipe' in locals():
  del pipe
if 'trainer' in locals():
  del trainer
import gc
gc.collect()
gc.collect()


# Reload the base model in FP16
base_model = AutoModelForCausalLM.from_pretrained(
    model_id,
    low_cpu_mem_usage=True,
    return_dict=True,
    torch_dtype=torch.float16,
    device_map=device_map
)

# Assuming PeftModel is a custom or previously defined model class for handling post-training operations
model = PeftModel.from_pretrained(base_model, new_model)
model = model.merge_and_unload()

# Reload tokenizer consistent with the first step and apply the same configurations
tokenizer = AutoTokenizer.from_pretrained(model_id)

# The following lines were commented out in the first step but included here for consistency
# Uncomment and adjust if necessary based on your specific requirements
# tokenizer.add_special_tokens({'pad_token': '[PAD]'})
# model.resize_token_embeddings(len(tokenizer))

# This setting was not changed in the first step, but keep it if needed for your use case
tokenizer.padding_side = "right"

In [ ]:
# Quantize
QUANTIZATION_METHODS = ["q4_k_m", "q5_k_m"]

# Convert to fp16
fp16 = f"{new_model}.fp16.bin"
model_path = os.path.join(new_model, fp16)
print(model_path)
!python llama.cpp/convert.py {new_model} --outtype f16 --outfile {model_path}

# Quantize the model for each method in the QUANTIZATION_METHODS list
for method in QUANTIZATION_METHODS:
    qtype = f"{new_model}/{new_model}.{method.upper()}.gguf"
    !llama.cpp\quantize {model_path} {qtype} {method}

In [ ]:
# Update Modelfile
import subprocess
command = ["ollama", "create", "TRACHI", "-f", "modelfiles/Modelfile_TRACHI"]
result = subprocess.run(command, capture_output=True, text=True, encoding='utf-8')

# Print stdout and stderr
print("Standard Output:\n", result.stdout)
if result.stderr:
    print("Standard Error:\n", result.stderr)

In [25]:
# Push -> HF
from huggingface_hub import create_repo, HfApi
api = HfApi()

# Upload gguf files
api.upload_folder(
    folder_path=new_model,
    repo_id=f"norygano/{new_model}-GGUF",
    allow_patterns=f"*.gguf",
    repo_type="model",
)

dolphin-mistral-TRACHI-7b.Q5_K_M.gguf:   0%|          | 0.00/5.13G [00:00<?, ?B/s]

dolphin-mistral-TRACHI-7b.Q4_K_M.gguf:   0%|          | 0.00/4.37G [00:00<?, ?B/s]

Upload 2 LFS files:   0%|          | 0/2 [00:00<?, ?it/s]

CommitInfo(commit_url='https://huggingface.co/norygano/dolphin-mistral-TRACHI-7b-GGUF/commit/80ec6feba5b7b321abaf0cca62cd244c1639ea1f', commit_message='Upload folder using huggingface_hub', commit_description='', oid='80ec6feba5b7b321abaf0cca62cd244c1639ea1f', pr_url=None, pr_revision=None, pr_num=None)

In [ ]:
# DEBUG: Push
import locale
locale.getpreferredencoding = lambda: "UTF-8"

model.push_to_hub(new_model, use_temp_dir=False)
tokenizer.push_to_hub(new_model, use_temp_dir=False)

In [ ]:
# DEBUG: Inference
from transformers import pipeline

model.eval()

# Set the prompt from user input
prompt = "Who are you?"

# Specify the character or context you want to prompt
character_name = "Ganymede"

# Initialize the text-generation pipeline with your fine-tuned model
pipe = pipeline(task="text-generation", model=model, tokenizer=tokenizer, max_length=200)

# Format the prompt to conform with ChatML template and include the system layer for the character
formatted_prompt = f"<conversation>\n  <system>{character_name}</system>\n  <exchange>\n    <user>{prompt}</user>\n    <assistant>"

# Generate the response using the pipeline
result = pipe(formatted_prompt)

# Extract the generated text. It's important to handle the output correctly based on how your model appends its response.
# Assuming the model generates the closing tags automatically. Adjust based on your model's behavior.
generated_text = result[0]['generated_text']

# Optionally, you might want to process `generated_text` to extract only the assistant's response.
# This processing step will depend on how the generated text structures the assistant's response and any closing tags.

print(generated_text)

In [ ]:
# DEBUG: Empty VRAM
if 'new_model' in locals():
  del model
if 'pipe' in locals():
  del pipe
if 'trainer' in locals():
  del trainer
import gc
gc.collect()
gc.collect()

In [ ]:
# DEBUG: Iterate through model parameters + print data types
for name, param in model.named_parameters():
    print(f"Layer: {name} | Size: {param.size()} | Type: {param.dtype}")

In [ ]:
# DEBUG: Check Files
import os
import glob

# Construct the pattern to match .gguf files
pattern = os.path.join(new_model, '*.gguf')

print(pattern)

# Use glob to find all files in the directory that match the pattern
gguf_files = glob.glob(pattern)

# Iterate over the list of gguf files and print each one
for file_path in gguf_files:
    print(file_path)

In [ ]:
# DEBUG: CUDA capability
import torch
torch.cuda.is_available()

In [ ]:
# DEBUG: Install Pytorch & other libraries

#pip install "torch==2.1.2" tensorboard

# Install Hugging Face libraries
!pip install  --upgrade \
  "transformers==4.38.2" \
  "datasets==2.16.1" \
  "accelerate==0.26.1" \
  "evaluate==0.4.1" \
  "bitsandbytes==0.41.1" \
  "trl==0.7.11" \
  "peft==0.8.2"